# 01. 模型 [LangChain Model](https://python.langchain.com/en/latest/modules/models/getting_started.html)

LangChain 三种最常用的模型

+ `LLM`: 大语言模型
+ `Chat Model` 聊天模型，经过微调 后的 LLM
+ `Text Embedding Model` 文本嵌入模型

## 1. LLM

+ 输入 字符串，输出 字符串
+ 例子: GPT-3, text-davinci-003

下面 加载 LLM 模型 `text_davinci-003`

前提是 先安装 `openai` 和 `langchain` 这两个`python`库

In [2]:
from langchain.llms import OpenAI

# 如果 你的 环境变脸有 OPENAI_API_KEY 则不需要填写 api_key 参数；
# 注：因为用的 LLM，所以这里不能配置 ChatGPT / GPT-4，因为这两个是 Chat Model
llm = OpenAI(
    client="davinci-api", 
    model="text-davinci-003", 
    max_tokens=256,  # 最大Token数
    temperature=0.5, # 温度：[0, 1]，越大越随机
)

然后 让 LLM “接龙” 下一个句子

In [9]:
# 等同于 llm.predict("say hi!")

llm("say hi!")

'\n\nHi there!'

也可以尝试 用 llm 来 处理 Message

注：内部实现，会将 message 换成 一堆提示 的 环境 字符串；

In [7]:
from langchain.schema import HumanMessage

llm.predict_messages([HumanMessage(content="say hi!")])

AIMessage(content='\n\nRobot: Hi there!', additional_kwargs={}, example=False)

用 generate 接口，取到 一些内部信息，比如消耗token的数目 之类

In [16]:
# 先生成 输入列表，重复15次 输入的内容

array = 5 * ["说个笑话给我听", "告诉我一首唐诗"]

print(f"数组长度为 = {len(array)}, 内容是 = {array}")

数组长度为 = 10, 内容是 = ['说个笑话给我听', '告诉我一首唐诗', '说个笑话给我听', '告诉我一首唐诗', '说个笑话给我听', '告诉我一首唐诗', '说个笑话给我听', '告诉我一首唐诗', '说个笑话给我听', '告诉我一首唐诗']


用 上面列表作为参数 调用 `llm.generate`

In [17]:
llm_result = llm.generate(array)

print(f"生成的结果长度为 = {len(llm_result.generations)}")


生成的结果长度为 = 10


看看第一个内容，是不是一个笑话

In [18]:
llm_result.generations[0]

[Generation(text='\n\n两只熊在森林里走着，突然一只熊说：“我觉得我们刚才走了一圈！”另一只熊说：“哦，你是不是太过熊熊了？”', generation_info={'finish_reason': 'stop', 'logprobs': None})]

最后一个是唐诗？

In [19]:
llm_result.generations[-1]

[Generation(text='\n\n《江南》\n\n江南可采莲，莲叶何田田。\n鱼戏莲叶间，青青江水田。\n田田江水头，青青河桥边。\n河桥垂杨柳，柳色向江天。', generation_info={'finish_reason': 'stop', 'logprobs': None})]

输出 所有内容看看 

In [20]:
print(llm_result.generations)

[[Generation(text='\n\n两只熊在森林里走着，突然一只熊说：“我觉得我们刚才走了一圈！”另一只熊说：“哦，你是不是太过熊熊了？”', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n《静夜思》\n\n床前明月光，\n疑是地上霜。\n举头望明月，\n低头思故乡。', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n两个熊在森林里走，一只熊突然停下来，另一只熊问：“怎么了？” 第一只熊回答：“我刚刚忘记我在哪里了！”', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n《静夜思》\n\n床前明月光，\n疑是地上霜。\n举头望明月，\n低头思故乡。', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n两个熊在森林里走着，突然一只熊对另一只说：“我刚吃了一只小猪，味道真不错！”另一只熊说：“哦？我没吃过，你能把它变小一点给我吃吗？”', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n《静夜思》\n\n床前明月光，\n疑是地上霜。\n举头望明月，\n低头思故乡。', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n两个人在餐厅吃饭，服务员问他们：“你们要什么？”\n\n一个人说：“我想要一份牛排。”\n\n另一个人说：“我想要一份薯条。”\n\n服务员说：“你们两个像牛排和薯条，真是一对恩爱的夫妻！”', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Ge

看看消耗了多少 Token

In [21]:
llm_result.llm_output

{'token_usage': {'completion_tokens': 1208,
  'total_tokens': 1363,
  'prompt_tokens': 155},
 'model_name': 'text-davinci-003'}

觉得贵的话 ，可以事先查下 问题要多少 token

先 安装 tiktoken 库

``` bash
pip3 install tiktoken
```

In [9]:
# 先用 pip3 install tiktoken 安装 tiktoken 库

input = "说个笑话给我听吧"

token = llm.get_num_tokens(input)

print(f'"{input}" 的 token 数为 {token}')

"说个笑话给我听吧" 的 token 数为 18


## 2. Chat Model 聊天模型

+ 输入: Messsage列表, 输出 Message
+ 例子: ChatGPT, GPT-4


In [3]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(
    client="chat", 
    model="gpt-3.5-turbo", # 可以改成 "gpt-4" 
    max_tokens=256, 
    temperature=0.5
)

照样可以 用 ChatMessage 接龙

In [11]:
chat_model.predict("say hi!")

'Hi there! How can I assist you today?'

试试 Message，这个更接近 原始 ChatGPT 的 API

In [8]:
chat_model.predict_messages([HumanMessage(content="say hi!")])

AIMessage(content='Hi there! How can I assist you today?', additional_kwargs={}, example=False)

加载几个 预定义 消息模式： 

+ `AIMessage` AI回复的消息类型
+ `HumanMessage` 人类问问题的消息类型 
+ `SystemMessage` 系统提示，给出AI的一些角色

In [4]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

直接聊天吧

In [6]:
chat_model([
    HumanMessage(content="翻译以下这句话成英文：我爱学习，学习是我妈快乐！我妈快乐，我全家都快乐！")
])

AIMessage(content='I love studying, and studying brings happiness to my mother. When my mother is happy, my whole family is happy!', additional_kwargs={}, example=False)

Chat Model 支持 多个 输入：

In [7]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates Chinese to English."),

    HumanMessage(content="我爱学习，学习是我妈快乐！我妈快乐，我全家都快乐！")
]

chat_model(messages)

AIMessage(content='I love learning, learning brings happiness to my mother! When my mother is happy, my whole family is happy!', additional_kwargs={}, example=False)

用 generate 接口，能得到更多信息：

留意输出的 token信息，用了多少token

In [8]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love artificial intelligence.")
    ],
]

result = chat_model.generate(batch_messages)

result

LLMResult(generations=[[ChatGeneration(text="J'adore la programmation.", generation_info=None, message=AIMessage(content="J'adore la programmation.", additional_kwargs={}, example=False))], [ChatGeneration(text="J'adore l'intelligence artificielle.", generation_info=None, message=AIMessage(content="J'adore l'intelligence artificielle.", additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 57, 'completion_tokens': 20, 'total_tokens': 77}, 'model_name': 'gpt-3.5-turbo'})

### 2.1 提示模板 `Prompt Template`

多个 `MessagePromptTemplate` 组成 `ChatPromptTemplate` 

下面导入 模板：

In [9]:
from langchain import PromptTemplate

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

简单的使用：

In [10]:
# 定义 系统提示
template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

# 定义 人类 提示
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [11]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 获得 回应，三个参数对应 上面模板的三个 {}
chat_model(chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.").to_messages())

AIMessage(content="J'adore la programmation.", additional_kwargs={}, example=False)

### 2.2. `LLMChain` 链

可以通过 链 将 llm 和 prompt 链接起来，之后还会看到更多的参数 练成一个整体；

In [12]:
from langchain import LLMChain

chain = LLMChain(llm=chat_model, prompt=chat_prompt)

In [13]:
chain.run(input_language="English", output_language="French", text="I love programming.")

"J'adore la programmation."

### 2.3. 流式响应 `Stream` 

OpenAI 的 ChatGPT API 可以 流式的方式，一个字符一个字符的回答，LangChain 也封装了这个。 

In [14]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

chat = ChatOpenAI(client="chat", streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0)

resp = chat([HumanMessage(content="写一首描述六一儿童节的唐诗")])

六一儿童节，欢声笑语乐无边。
小朋友们手牵手，共同度过这美好的天。
彩旗飘扬，气球飞舞，热闹非凡。
游戏玩具，糖果零食，满足了孩子们的心愿。
家长们陪伴左右，关爱呵护无微不至。
让孩子们快乐成长，健康茁壮，是我们的使命。
六一儿童节，让我们一起欢庆，祝福所有的孩子们。

## 3. `Text Embedding Model` 文本嵌入模型

输入 字符串，输出浮点数 数组，代表输入串的`嵌入向量`。

两个向量之间的距离 衡量 它们的相关性。

小距离表示高相关性，大距离表示低相关性。

主要用于：

+ 搜索（其中结果按与查询字符串的相关性排名）
+ 聚类分析（其中文本字符串按相似性分组）
+ 建议（建议使用具有相关文本字符串的项目）
+ 异常检测（识别相关性不大的异常值）
+ 多样性测量（分析相似性分布）
+ 分类（其中文本字符串按其最相似的标签分类）

下面 关注 OpenAI的 Embedding API

In [15]:
from langchain.embeddings import OpenAIEmbeddings

In [27]:
embeddings = OpenAIEmbeddings(client="api", model="text-embedding-ada-002")

In [28]:
text = "This is a test document."

In [29]:
query_result = embeddings.embed_query(text)

query_result

[-0.003158408682793379,
 0.011094409972429276,
 -0.0040013170801103115,
 -0.011747414246201515,
 -0.0010153218172490597,
 0.010781234130263329,
 -0.010368109680712223,
 -0.005297331139445305,
 -0.009881687350571156,
 -0.02616015262901783,
 0.020376399159431458,
 0.022575292736291885,
 -0.007522876374423504,
 0.01728462427854538,
 -0.006003641523420811,
 0.01912369765341282,
 0.02125595696270466,
 -0.015645449981093407,
 0.007669468875974417,
 -0.018364081159234047,
 -0.0006909018848091364,
 -0.006416766904294491,
 -0.01098113413900137,
 0.017950955778360367,
 -0.02213551290333271,
 -0.003076783148571849,
 0.01437942124903202,
 -0.029984891414642334,
 0.01852400042116642,
 -0.007916011847555637,
 0.010068260133266449,
 -0.019456863403320312,
 -0.003688141703605652,
 -0.024241119623184204,
 -0.005530546884983778,
 0.003824739484116435,
 -0.005653818137943745,
 -0.029451826587319374,
 0.018603960052132607,
 -0.016618292778730392,
 0.00752953952178359,
 0.012420408427715302,
 -0.0009903343

In [20]:
# documents = [句子1, 句子2, ...]
doc_result = embeddings.embed_documents([text])

doc_result

[[-0.013454702504113568,
  -0.007768793394064358,
  -0.013286609957498653,
  -0.03537249145620518,
  -0.01126950032944223,
  0.0197325886400654,
  -0.008601946642009136,
  0.00100033263278741,
  -0.023620640545323757,
  -0.013776270489728926,
  0.029365015757524454,
  0.025184631013202247,
  -0.001400465709185438,
  0.0018051664570900086,
  -0.0003325306708070621,
  -0.001953160829980984,
  0.026573219914997306,
  -0.006040364889305211,
  0.01701387740428805,
  -0.0016407281873787117,
  -0.01210265497437085,
  0.007070844433432571,
  0.0062121114799931045,
  -0.006475212432134411,
  0.003297900271577745,
  -0.009420484179323283,
  0.022392833308029974,
  -0.015742221638118534,
  -0.011320659740431605,
  -0.004922184562136135,
  -0.01683847583820462,
  -0.010078237258168473,
  0.019337936979022797,
  -0.015157551303378028,
  -0.02283133536059343,
  -0.004037872368863262,
  0.013973596320250225,
  -0.017934730038290703,
  0.02739175745231145,
  -0.008009969150445235,
  -0.009179307957281

看下 另一套接口

In [21]:
from langchain.embeddings.openai import OpenAIEmbeddings

In [22]:
embeddings = OpenAIEmbeddings(client="api")

In [23]:
text = "This is a test document."

In [24]:
query_result = embeddings.embed_query(text)

query_result

[-0.0031265460420399904,
 0.01113363541662693,
 -0.004037691745907068,
 -0.011746617965400219,
 -0.0009935986017808318,
 0.010807155631482601,
 -0.010440697893500328,
 -0.005253663286566734,
 -0.009874355047941208,
 -0.02617170475423336,
 0.02034836821258068,
 0.022573761641979218,
 -0.007522366475313902,
 0.01723014935851097,
 -0.005986577831208706,
 0.0191490538418293,
 0.021254515275359154,
 -0.015644390136003494,
 0.0076422980055212975,
 -0.018402813002467155,
 -0.0006866907933726907,
 -0.006416331976652145,
 -0.010967063717544079,
 0.018003040924668312,
 -0.022173991426825523,
 -0.0030682459473609924,
 0.014405098743736744,
 -0.029982859268784523,
 0.018496092408895493,
 -0.007935463450849056,
 0.010047589428722858,
 -0.01944221928715706,
 -0.0036945545580238104,
 -0.024279452860355377,
 -0.005540166050195694,
 0.003831143258139491,
 -0.005666760727763176,
 -0.029449831694364548,
 0.01857604645192623,
 -0.016643818467855453,
 0.007529029157012701,
 0.01245288085192442,
 -0.0009711

In [25]:
doc_result = embeddings.embed_documents([text])

doc_result

[[-0.0031265460635144116,
  0.011133635493097378,
  -0.004037691773639618,
  -0.011746618046080886,
  -0.000993598608605281,
  0.01080715570571065,
  -0.010440697965211392,
  -0.005253663322651083,
  -0.009874355115762394,
  -0.026171704933991537,
  0.020348368352341755,
  0.02257376179702522,
  -0.00752236652698065,
  0.017230149476854816,
  -0.005986577872327018,
  0.01914905397335298,
  0.021254515421344024,
  -0.015644390243455687,
  0.007642298058011785,
  -0.018402813128865346,
  -0.0006866907980891692,
  -0.006416332020722187,
  -0.010967063792870444,
  0.0180030410483207,
  -0.022173991579125737,
  -0.0030682459684349844,
  0.014405098842676966,
  -0.029982859474719296,
  0.018496092535934365,
  -0.007935463505353126,
  0.01004758949773389,
  -0.019442219420694323,
  -0.003694554583399551,
  -0.024279453027116777,
  -0.005540166088247863,
  0.0038311432844533796,
  -0.00566676076668485,
  -0.029449831896638266,
  0.018576046579514262,
  -0.016643818582172135,
  0.00752902920872